<a href="https://colab.research.google.com/github/mylovepyd03/26-1-Lab-Intern/blob/main/Resnet_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet

In [1]:
import torch
from torch import Tensor
import torch.nn as nn

In [2]:
class BasicBlock(nn.Module):#conv2개
    expansion_factor = 1
    def __init__(self, in_channels: int, out_channels: int, stride: int = 1):
      super(BasicBlock, self).__init__() #상속nn module 기능세팅

       #층설계
      self.stride = stride
      self.in_channels= in_channels
      self.out_channels = out_channels

        # 첫 번째 레이어: 이미지 크기를 stride에 맞춰 조절
      self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
      self.bn1 = nn.BatchNorm2d(out_channels) #배치정규화가 나오므로 bias 를 false로 설정
      self.relu = nn.ReLU(inplace=True)

        # 두 번째 레이어: 채널과 크기 유지
      self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
      self.bn2 = nn.BatchNorm2d(out_channels)

        # 지름길(Shortcut): 입력 x가 출력과 모양이 다를 때(stride가 있거나 채널이 바뀔 때) 크기를 맞춰줌
      self.shortcut = nn.Sequential()
      if stride != 1 or in_channels != out_channels: #입력과 출력의 구격이 다를경우
          self.shortcut = nn.Sequential(
              nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
              nn.BatchNorm2d(out_channels) #skip하는거, 그대로 데이터 데려감 이때 규격맞추어주기 위함ㅊ채널,규격 다)
          )

    def forward(self, x: Tensor) -> Tensor:

        identity = x #원래 데이터 복사:입력정보x를 받아와서 identuty변수에 따로 빼둔다.

        # 2. 메인 경로 연산 (Conv -> BN -> ReLU -> Conv -> BN) #데이터가공
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # 3. 지름길 더하기 (잔차 연결)
        # 만약 크기가 다르다면 shortcut()을 거쳐서 크기를 맞춘 뒤 더함
        out += self.shortcut(identity) #가공 결과에 원래 데이터를 정해서 정보의소실을 막는것

        # 4. 마지막 ReLU
        out = self.relu(out)
        return x


In [3]:
class BottleNeck(nn.Module):
#conv3개

    expansion_factor = 4 # 표준 레즈넷크기 반영
    def __init__(self, in_channels: int, out_channels: int, stride: int = 1):
        super(BottleNeck, self).__init__()
        self.in_channels= in_channels
        # 1. 첫 번째 층: 1x1 Conv (채널을 좁게 줄여서 계산량을 줄임) 256번만 계산하면돼서 훨씬 계산이 빨라서 잘줄여줌
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        # 2. 두 번째 층: 3x3 Conv (좁아진 채널 상태로 특징 추출)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # 3. 세 번째 층: 1x1 Conv (채널을 다시 넓게 확장 - expansion_factor 배)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion_factor,
                               kernel_size=1, bias=False)#줄어든값을 다시 늘려주는 역할 ,(64x4)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion_factor)

        self.relu = nn.ReLU(inplace=True)

        # 4. 지름길(Shortcut): 입력 x와 출력의 모양이 다를 때 맞춰줌
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion_factor:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion_factor,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion_factor)
            )


    def forward(self, x:Tensor) -> Tensor: #실제 행동

        identity = x

        # 첫 번째: 1x1 Conv로 채널 압축 (다이어트)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        # 두 번째: 3x3 Conv로 진짜 공부 (특징 추출)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        # 세 번째: 1x1 Conv로 채널 확장 (다시 뻥튀기)
        out = self.conv3(out)
        out = self.bn3(out)

        # 3. 마법의 합치기 (Shortcut Connection)
        # 공부한 결과(out)에 아까 챙겨둔 원래 데이터(identity)를 더합니다.
       #규격은 알아서 아까위에 코드에서 맞춤 (init)
        out += self.shortcut(identity)

        # 마지막으로 ReLU를 한 번 더 거치며 마무리!
        out = self.relu(out)

        return out


In [4]:
#코드내가짠거
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        self.block= block
        self.num_blocks = num_blocks
        self.num_classes = num_classes
        super(ResNet, self).__init__()
        self.in_channels = 64 # 초기 채널 설정

        # 1. 첫 번째 Conv 레이어: 입력 이미지(3채널)를 처리
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        # 2. 4개의 ResNet Layer (Stage) 생성
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        #stride를 크게 할수록 정보손실을 막기위해 채널의 수를 늘림

        # 3. 출력 레이어 (Global Average Pooling + Fully Connected)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion_factor, num_classes)

        self._init_layer()
      #블록 연결
    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion_factor
        return nn.Sequential(*layers)
       #세팅 최적화 가중치 초기값 설정하고,  데이터 정규화 해주는 장치들을 아무것도 건드리지 않는 상태로 맞춰주기
    def _init_layer(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x: Tensor) -> Tensor:
        # 초기 레이어 통과
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        # 각 Stage 통과
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # 분류 레이어 통과
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x) #결과판단


        return x


In [5]:
class Model:
    def resnet18(self, num_classes=10): # 기본값은 10으로 두되
      res18 = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

      return res18
        # 여기에 코드를 작성해주세요

    def resnet34(self):
        res34= ResNet(BasicBlock, [3, 4, 6, 3])

        return res34
        # 여기에 코드를 작성해주세요

    def resnet50(self):
        res50= ResNet(BottleNeck, [3, 4, 6, 3])

        return res50
        # 여기에 코드를 작성해주세요

    def resnet101(self):
        res101= ResNet(BottleNeck, [3, 4, 23, 3])

        return res101
        # 여기에 코드를 작성해주세요

    def resnet152(self):
        res152= ResNet(BottleNeck, [3, 8, 36, 3])
        return res152
        # 여기에 코드를 작성해주세요


In [6]:
model = Model().resnet152()
y = model(torch.randn(1, 3, 224, 224))
print(y.size())


torch.Size([1, 10])


In [7]:
import os
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# hyper-parameters
learning_rate = 0.01
weight_decay = 1e-4
momentum = 0.9
batch_size = 128
num_epochs = 1

In [8]:
def do_transform(train_mean, train_std, test_mean, test_std):
    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(train_mean, train_std),
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.Normalize(test_mean, test_std),
    ])

    return train_transform, test_transform

def do_mean_std(train_data, test_data):
    train_mean_rgb = [np.mean(x.numpy(), axis=(1,2)) for x, _ in train_data]
    train_std_rgb  = [np.std(x.numpy(), axis=(1,2)) for x, _ in train_data]

    test_mean_rgb = [np.mean(x.numpy(), axis=(1,2)) for x, _ in test_data]
    test_std_rgb  = [np.std(x.numpy(), axis=(1,2)) for x, _ in test_data]

    train_mean = np.mean(train_mean_rgb, axis=0).tolist()
    train_std  = np.mean(train_std_rgb, axis=0).tolist()
    test_mean  = np.mean(test_mean_rgb, axis=0).tolist()
    test_std   = np.mean(test_std_rgb, axis=0).tolist()

    return train_mean, train_std, test_mean, test_std


In [9]:
#내가수정
def get_dataloader(train_data, test_data, batch_size=64):
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    # 여기에 코드를 작성해주세요
    return train_loader, test_loader

In [10]:
train_data = datasets.STL10(
    root='./data',
    split='train',
    download=True,
    transform=transforms.ToTensor() #사진(이미지 파일)을 AI가 계산할 수 있는 숫자(Tensor) 형태로 바로 바꿔서 가져와라는 뜻
)

test_data = datasets.STL10(
    root='./data',
    split='test',
    download=True,
    transform=transforms.ToTensor()
)


In [11]:
train_mean, train_std, test_mean, test_std = do_mean_std(train_data, test_data)
train_transform, test_transform = do_transform(train_mean, train_std, test_mean, test_std)

train_data.transform = train_transform
test_data.transform  = test_transform

train_loader, test_loader = get_dataloader(train_data, test_data)

In [12]:
np.random.seed(123)
torch.manual_seed(123)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [13]:
model = Model().resnet152().to(device)
learning_rate = 0.001  # 공부 속도 (너무 크면 성적이 튀고, 작으면 너무 느림)
weight_decay = 1e-4    # 과적합 방지
criterion = nn.CrossEntropyLoss()

import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
# 학습률 조절


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    # -------- Train --------
    model.train()
    correct, count = 0, 0
    train_loss = 0.0

    for step, (images, labels) in enumerate(train_loader, start=1):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        train_loss= criterion(model(images), labels)
        train_loss.backward()
        optimizer.step()

        count += len(labels)
        correct += (model(images).argmax(dim=1) == labels).sum().item()

        train_loss += loss.item()

        print(
            f"[Train] Epoch {epoch} "
            f"Step {step}/{len(train_loader)} "
            f"Acc {(correct/count)*100:.2f}% "
            f"Loss {(train_loss/count):.4f}"
        )

    # -------- Validation --------
    model.eval()
    correct, count = 0, 0
    valid_loss = 0.0

    with torch.no_grad():
        for step, (images, labels) in enumerate(test_loader, start=1):
            images, labels = images.to(device), labels.to(device)
            #테스트용 예측과 오차 계산
            outputs = model(images)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()
            count += len(labels)
            correct += (outputs.argmax(dim=1) == labels).sum().item()

            print(
                f"[Valid] Step {step}/{len(test_loader)} "
                f"Acc {(correct/count)*100:.2f}% "
                f"Loss {(valid_loss/count):.4f}"
            )

    scheduler.step(valid_loss) ## 성적이 안 좋으면 학습률을 낮추는것